# **TEXT PREPROCESSING FUNCTION**

In [25]:
import pandas as pd
import re #regular expressions library for text manipulation
import string
import numpy as np
import unicodedata

import csv

from bs4 import BeautifulSoup
import html2text

from glob import glob

#NLP libraries
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy import displacy
from spacy.lang.en import English
import spacymoji
import emoji
import contractions

import itertools
from autocorrect import Speller

#for wordclouds
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

In [2]:
def clean_comments (filepath):
    #import data
    df = pd.read_csv(filepath, low_memory = False)
    
    #remove deleted comments if any
    df = df[df.Body != '[deleted]']
    
    #remove comments with missing id
    drop_index = df[df.isSubmitter.isnull()].index
    df.drop(drop_index, inplace = True)
    
    #remove duplicates if any
    df = df.drop_duplicates(subset =['ID'], ignore_index = True)
    df.reset_index (drop = True, inplace = True)
    
    #correct data types and column label
    df['Date_Created'] = pd.to_datetime(df['Date_Created'])
    df['year'] = df['Date_Created'].dt.year
    df['Score'] = df['Score'].astype('int') 
    df.rename(columns = {'Author_ID': "Author"}, inplace = True) 
    
    #remove unnecessary columns
    df.drop(columns = ['Unnamed: 0', 'Author', 'Score', 'Date_Created',
       'Parent_ID', 'Submission_ID', 'Subreddit', 'isParent', 'isSubmitter'], inplace = True)
    
    #remove any extra whitespace in column labels
    df.columns = df.columns.str.strip()
    
    #include column to denote row is comment entry
    df['text_type'] = 'comment'
    
    #rearrange column order
    df = df[['text_type','ID', 'year', 'Body']]
    
    #rename 'Body' column to text
    df.rename(columns = {'Body': 'long_text'}, inplace = True)
        
    return df



In [3]:
def clean_submissions(filepath):
    df = pd.read_csv(filepath, low_memory = False)
    
    #drop duplicate posts
    df = df.drop_duplicates(subset =['ID'], ignore_index = True)
    df.reset_index (drop = True, inplace = True)
    
    #create column for post title + post text
    df['long_text'] = df['Title']+ " " +df['Post Text'].fillna('')
    
    #adjust data types
    df['Date Created'] = pd.to_datetime(df['Date Created'])
    df['year'] = df['year'].astype('int')
    
    #remove unwanted columns
    df = df.drop(columns = ['Unnamed: 0', 'Title','Post Text', 'Date Created', 'Score',
       'Total Comments', 'Post URL', 'SubReddit','Unnamed: 0.1'])
    
    #include column to denote row is comment entry
    df['text_type'] = 'submission'
    
    #reorder columns - 'ID', 'Post Text'
    df  = df[['text_type','ID', 'year', 'long_text']]
    
    return df
    

In [4]:
comments_filepath = 'Data/comments.csv'
submissions_filepath = 'Data/full_posts.csv'


data = pd.concat([clean_comments(comments_filepath), clean_submissions(submissions_filepath)], ignore_index = True)

data

,text_type,ID,year,long_text
0,comment,gtfo2hl,2021,"*Cuntry roads, take me hoem*"
1,comment,gtfqkbv,2021,"That’s been there for several years, sent a pi..."
2,comment,gtfou07,2021,I am single and I have not traveled to any cun...
3,comment,gtfrgpe,2021,What happens when you shop at dragon mart...
4,comment,gtg5mwv,2021,I am cunting on them to do so 😅
...,...,...,...,...
130050,submission,14f46ji,2023,"Best beauty saloons in Dubai? Hello fellas, I ..."
130051,submission,14f4uyi,2023,Found the r/dubai redditors who kept telling m...
130052,submission,14f4ri3,2023,Scam ? Healthy.line My sister has a CBD debit ...
130053,submission,14f4k3r,2023,Thoughts on Expo City properties? Anyone else ...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130055 entries, 0 to 130054
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text_type  130055 non-null  object
 1   ID         130055 non-null  object
 2   year       130055 non-null  int64 
 3   long_text  130055 non-null  object
dtypes: int64(1), object(3)
memory usage: 4.0+ MB


In [6]:
year_group = data.groupby(by='year')
for year, group in year_group:
    print (year,len(group))

2012 11
2013 60
2014 143
2015 206
2016 787
2017 1131
2018 1686
2019 4238
2020 29987
2021 28188
2022 36776
2023 26842


## **TEXT PREPROCESSING**

### **Convert all Text to Lowercase**

In [7]:
data['clean_text'] = data['long_text'].apply(lambda text: text.lower())

data.sample(n=5)

,text_type,ID,year,long_text,clean_text
38197,comment,hrc3ct5,2022,"No, I actually agree with you. I remember in c...","no, i actually agree with you. i remember in c..."
62211,comment,fr9erp4,2020,The AC company?,the ac company?
33410,comment,jgwjm4h,2023,Good luck on your new path bro,good luck on your new path bro
89980,comment,iqqhx7n,2022,My take is most Asian or developing nation wor...,my take is most asian or developing nation wor...
64381,comment,itb9579,2022,"Also, the [minister of tolerance](http://www.t...","also, the [minister of tolerance](http://www.t..."


## **Insert Word Count**

In [8]:
data['word_count'] = data['clean_text'].apply (lambda text: len(text.split()))

data.sort_values(by='word_count', ascending = False)

,text_type,ID,year,long_text,clean_text,word_count
126269,submission,gzl2ec,2020,"PSA: Immigration to Canada, Australia, NZ Hell...","psa: immigration to canada, australia, nz hell...",3188
126556,submission,jh5hjq,2020,List of discounts and working promo codes in D...,list of discounts and working promo codes in d...,1945
126510,submission,tce5mm,2022,Unknown accident and the plight with Dubai Pol...,unknown accident and the plight with dubai pol...,1926
127807,submission,10kwnuo,2023,A complete list of LIFE HACK / TIPS / FACTS ab...,a complete list of life hack / tips / facts ab...,1593
110203,comment,fpg1e6j,2020,Here is the text for those that can’t get by t...,here is the text for those that can’t get by t...,1352
...,...,...,...,...,...,...
119467,comment,hsr1vcj,2022,Ok,ok,1
119468,comment,hsr2kg6,2022,Ok,ok,1
119469,comment,hsr3ak1,2022,Ok,ok,1
11417,comment,gectffr,2020,Superb.,superb.,1


## **Translate Emoticons**

In [9]:
"""def find_emoji(text):
    try:
        emoticon_details = next(emoji.analyze(text, join_emoji=True))
        emoticon = emoticon_details.chars
        #translated_emoji = emoji.demojize(emoticon.chars)
    except StopIteration:
        emoticon = ''  # Handle the case when no emoji is found
    return  emoticon

data['emoticons'] = data['long_text'].apply (lambda text: find_emoji(text) )
data['translated_emojis'] = data['emoticons'].apply(lambda text: emoji.demojize(text))

emoji_index = [3709,33734,129114,100878]

data.loc[emoji_index]"""

"def find_emoji(text):\n    try:\n        emoticon_details = next(emoji.analyze(text, join_emoji=True))\n        emoticon = emoticon_details.chars\n        #translated_emoji = emoji.demojize(emoticon.chars)\n    except StopIteration:\n        emoticon = ''  # Handle the case when no emoji is found\n    return  emoticon\n\ndata['emoticons'] = data['long_text'].apply (lambda text: find_emoji(text) )\ndata['translated_emojis'] = data['emoticons'].apply(lambda text: emoji.demojize(text))\n\nemoji_index = [3709,33734,129114,100878]\n\ndata.loc[emoji_index]"

## **Expand Word Contractions**

In [10]:
data['clean_text'] = data['clean_text'].apply(lambda text: contractions.fix(text)) 

data.sample(n=5)

,text_type,ID,year,long_text,clean_text,word_count
111643,comment,fdjnwg5,2020,Yup! its top-up time boys,yup! its top-up time boys,5
101508,comment,g0jszbe,2020,I don't understand did he do something wrong?,i do not understand did he do something wrong?,8
26031,comment,gbmcup8,2020,"hi again, im the deleted user (kentuckyboi69's...","hi again, i am the deleted user (kentuckyboi69...",46
120805,comment,iqwnydw,2022,"Well, claiming that the UAE segregates locals ...","well, claiming that the uae segregates locals ...",111
7709,comment,hs6aa6c,2022,look at the downvotes !😂,look at the downvotes !😂,5


## **Remove URLs**

In [11]:
pd.reset_option('display.max_colwidth')

In [12]:
#index of rows with urls
html_index = data[data['long_text'].str.contains("https")].index
data.loc[html_index]

,text_type,ID,year,long_text,clean_text,word_count
17,comment,gtgyqzz,2021,Meditation And Relaxation Music https://youtu....,meditation and relaxation music https://youtu....,5
83,comment,gpqwzwc,2021,I am curious about this Apollo Fintech GSX coi...,i am curious about this apollo fintech gsx coi...,21
94,comment,gpqwrek,2021,https://www.instagram.com/p/CKhzbwpH0c4/?igshi...,https://www.instagram.com/p/ckhzbwph0c4/?igshi...,7
174,comment,gq3tah4,2021,"With all due respect, do you expect a law enfo...","with all due respect, do you expect a law enfo...",96
293,comment,jb5kpz4,2023,Here: Dubai Festival City\nhttps://maps.app.go...,here: dubai festival city\nhttps://maps.app.go...,9
...,...,...,...,...,...,...
129949,submission,14djkvs,2023,Help‼️Uni qualification equivalency letter for...,help‼️uni qualification equivalency letter for...,81
130007,submission,14d2uk9,2023,PSA: Villas are at high risk of fire during th...,psa: villas are at high risk of fire during th...,106
130009,submission,14d0c29,2023,"Daily Random Discussion Thread Hello,\n\nThis ...","daily random discussion thread hello,\n\nthis ...",69
130040,submission,14cieby,2023,Need help with applying for residency - confus...,need help with applying for residency - confus...,135


In [13]:
#regex pattern for urls
url_pattern = r'https?://\S+'
#replace url with empty string
data['clean_text'] = data['clean_text'].apply(lambda text: re.sub(url_pattern, ' ', text, flags=re.MULTILINE))

data.loc[html_index]

,text_type,ID,year,long_text,clean_text,word_count
17,comment,gtgyqzz,2021,Meditation And Relaxation Music https://youtu....,meditation and relaxation music,5
83,comment,gpqwzwc,2021,I am curious about this Apollo Fintech GSX coi...,i am curious about this apollo fintech gsx coi...,21
94,comment,gpqwrek,2021,https://www.instagram.com/p/CKhzbwpH0c4/?igshi...,\n\nhere is a video explaining the same.,7
174,comment,gq3tah4,2021,"With all due respect, do you expect a law enfo...","with all due respect, do you expect a law enfo...",96
293,comment,jb5kpz4,2023,Here: Dubai Festival City\nhttps://maps.app.go...,here: dubai festival city\n \n\nthen follow th...,9
...,...,...,...,...,...,...
129949,submission,14djkvs,2023,Help‼️Uni qualification equivalency letter for...,help‼️uni qualification equivalency letter for...,81
130007,submission,14d2uk9,2023,PSA: Villas are at high risk of fire during th...,psa: villas are at high risk of fire during th...,106
130009,submission,14d0c29,2023,"Daily Random Discussion Thread Hello,\n\nThis ...","daily random discussion thread hello,\n\nthis ...",69
130040,submission,14cieby,2023,Need help with applying for residency - confus...,need help with applying for residency - confus...,135


## **Remove Accents from Characters**

In [14]:
data['clean_text'] = data['clean_text'].apply(
                                            lambda text: unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8'))

data.sample(n=5)

,text_type,ID,year,long_text,clean_text,word_count
25847,comment,hcc1h0q,2021,"However, there are no other benefits such as h...","however, there are no other benefits such as h...",31
118268,comment,igw1n6d,2022,We cheat by having aircon everywhere :D stay h...,we cheat by having aircon everywhere :d stay h...,11
91312,comment,jiiger9,2023,Screw that dude. Where I’m from people who low...,screw that dude. where i am from people who lo...,29
122564,comment,fponqk6,2020,backblaze is really good. I had used it once e...,backblaze is really good. i had used it once e...,62
112220,comment,g0xl5r0,2020,I’m not sure what the issue here??\n\nYou can ...,i am not sure what the issue here??\n\nyou can...,41


## **Remove Punctuations**

In [15]:
#index of some rows with punctuations
checker_list = ['ifquow','gzl2ec','147gsfl','vtelex',
 '12pqx6m','fuxrd2','2ui6wu','l4gz0u','14f4uyi','14f8d30']

rows_to_check = data[data['ID'].isin(checker_list)].index.tolist()

rows_to_check.extend([32003, 116022,18460,5786,30109])

rows_to_check.extend(html_index)

print(rows_to_check[:5])

[125800, 126269, 127899, 128041, 128388]


In [16]:
#regex pattern for punctuations
punctuation_pattern = r'[^\w\s]'

#remove punctuations using `re.sub() method
data['clean_text'] = data['clean_text'].apply(lambda text: re.sub(r'[^\w\s]', ' ', text))

data.iloc[rows_to_check]

,text_type,ID,year,long_text,clean_text,word_count
125800,submission,ifquow,2020,GUYS WE MADE IT!!! YAY,guys we made it yay,5
126269,submission,gzl2ec,2020,"PSA: Immigration to Canada, Australia, NZ Hell...",psa immigration to canada australia nz hell...,3188
127899,submission,147gsfl,2023,How to reach people who are asking for money/g...,how to reach people who are asking for money g...,72
128041,submission,12pqx6m,2023,How do you plan to spend your Eid holiday? Any...,how do you plan to spend your eid holiday any...,58
128388,submission,vtelex,2022,"Hi everyone, I'm currently looking for jobs te...",hi everyone i am currently looking for jobs t...,26
...,...,...,...,...,...,...
129949,submission,14djkvs,2023,Help‼️Uni qualification equivalency letter for...,help uni qualification equivalency letter for...,81
130007,submission,14d2uk9,2023,PSA: Villas are at high risk of fire during th...,psa villas are at high risk of fire during th...,106
130009,submission,14d0c29,2023,"Daily Random Discussion Thread Hello,\n\nThis ...",daily random discussion thread hello \n\nthis ...,69
130040,submission,14cieby,2023,Need help with applying for residency - confus...,need help with applying for residency confus...,135


## **Remove New Line & Tab**

In [17]:
#remove `\n` from text
data['clean_text'] = data['clean_text'].str.replace('\n', ' ')
#remove `\t` from text
data['clean_text'] = data['clean_text'].str.replace('\t', ' ')

data.iloc[rows_to_check]

,text_type,ID,year,long_text,clean_text,word_count
125800,submission,ifquow,2020,GUYS WE MADE IT!!! YAY,guys we made it yay,5
126269,submission,gzl2ec,2020,"PSA: Immigration to Canada, Australia, NZ Hell...",psa immigration to canada australia nz hell...,3188
127899,submission,147gsfl,2023,How to reach people who are asking for money/g...,how to reach people who are asking for money g...,72
128041,submission,12pqx6m,2023,How do you plan to spend your Eid holiday? Any...,how do you plan to spend your eid holiday any...,58
128388,submission,vtelex,2022,"Hi everyone, I'm currently looking for jobs te...",hi everyone i am currently looking for jobs t...,26
...,...,...,...,...,...,...
129949,submission,14djkvs,2023,Help‼️Uni qualification equivalency letter for...,help uni qualification equivalency letter for...,81
130007,submission,14d2uk9,2023,PSA: Villas are at high risk of fire during th...,psa villas are at high risk of fire during th...,106
130009,submission,14d0c29,2023,"Daily Random Discussion Thread Hello,\n\nThis ...",daily random discussion thread hello this is...,69
130040,submission,14cieby,2023,Need help with applying for residency - confus...,need help with applying for residency confus...,135


## **Remove Digits**

In [18]:
data['clean_text'] = data['clean_text'].apply(lambda text: ' '.join (word for word in text.split() if word.isalpha()))

## **Remove Stop Words - SpaCy**

In [19]:
nlp = spacy.load('en_core_web_md')

data['clean_text'] = data['clean_text'].apply(lambda text: ' '.join([word.text for word in nlp(text) if not word.is_stop]))

data.sample(n=10)

,text_type,ID,year,long_text,clean_text,word_count
48207,comment,jo5nyu9,2023,"Okay, you want a proper response?",okay want proper response,6
123195,comment,jkwletu,2023,Username...checks out?,username checks,2
77429,comment,invwata,2022,what was the issue. im curious to know what my...,issue curious know polish coworkers rentals lol,16
109947,comment,gftzmix,2020,Maybe they’re worried that there’s a finite su...,maybe worried finite supply free vaccine futur...,27
89554,comment,fyrlkjr,2020,"The National, you were supposed to fight the d...",national supposed fight dark join,13
112341,comment,in4zjvp,2022,As per you if you don't pray lord will not car...,pray lord care right op wants judge person bas...,49
59810,comment,iuuuj4f,2022,Your manager sounds like a real douche-bag. If...,manager sounds like real douche bag mind askin...,21
105425,comment,ig451vx,2022,I'm loving the comments on this post.,loving comments post,7
31716,comment,gy8rffk,2021,Not taking any sides here just wanted to point...,taking sides wanted point anti semetic rhetori...,90
112933,comment,ib4nisw,2022,This is the way 🙌🏽,way,5


## **Remove Extra Whitespaces**

In [20]:
data['clean_text'] = data['clean_text'].str.strip().str.replace('\s+', ' ', regex = True)

sample_rows = [5786,18460, 103391, 129297]

#redo word_count
data['word_count'] = data['clean_text'].apply (lambda text: len(text.split()))

data.loc[sample_rows].sort_values(by='word_count', ascending = False)

,text_type,ID,year,long_text,clean_text,word_count
5786,comment,ji1smg1,2023,"FULLTEXT\nApr 27, 2023\n\nStand in the middle ...",fulltext apr stand middle teeming meena bazaar...,659
18460,comment,jj2t0ih,2023,"FULL TEXT:\n\nBy Archana Narayanan, Abeer Abu ...",text archana narayanan abeer abu omar zainab f...,600
129297,submission,l4gz0u,2021,BEST Karak/ Zafrani/ Morrocan Tea in Dubai? He...,best karak zafrani morrocan tea dubai heya bes...,19
103391,comment,d5buby5,2016,That's very kind of you. :),kind,1


## **Word Tokenization - NLTK**

In [21]:
data['tokens'] = data['clean_text'].apply(lambda text: word_tokenize(text))

data.iloc[rows_to_check]

,text_type,ID,year,long_text,clean_text,word_count,tokens
125800,submission,ifquow,2020,GUYS WE MADE IT!!! YAY,guys yay,2,"[guys, yay]"
126269,submission,gzl2ec,2020,"PSA: Immigration to Canada, Australia, NZ Hell...",psa immigration canada australia nz hello duba...,1342,"[psa, immigration, canada, australia, nz, hell..."
127899,submission,147gsfl,2023,How to reach people who are asking for money/g...,reach people asking money grocery experience t...,26,"[reach, people, asking, money, grocery, experi..."
128041,submission,12pqx6m,2023,How do you plan to spend your Eid holiday? Any...,plan spend eid holiday fun activities trips li...,28,"[plan, spend, eid, holiday, fun, activities, t..."
128388,submission,vtelex,2022,"Hi everyone, I'm currently looking for jobs te...",hi currently looking jobs temporary permanent ...,12,"[hi, currently, looking, jobs, temporary, perm..."
...,...,...,...,...,...,...,...
129949,submission,14djkvs,2023,Help‼️Uni qualification equivalency letter for...,help uni qualification equivalency letter mast...,37,"[help, uni, qualification, equivalency, letter..."
130007,submission,14d2uk9,2023,PSA: Villas are at high risk of fire during th...,psa villas high risk fire summer months friend...,48,"[psa, villas, high, risk, fire, summer, months..."
130009,submission,14d0c29,2023,"Daily Random Discussion Thread Hello,\n\nThis ...",daily random discussion thread hello daily ran...,45,"[daily, random, discussion, thread, hello, dai..."
130040,submission,14cieby,2023,Need help with applying for residency - confus...,need help applying residency confused order st...,68,"[need, help, applying, residency, confused, or..."


## **LEMMATIZATION**

In [22]:
data['tokens'] = data['tokens'].apply(lambda text: [token.lemma_ for token in nlp(' '.join(text))])

data.iloc[rows_to_check]

,text_type,ID,year,long_text,clean_text,word_count,tokens
125800,submission,ifquow,2020,GUYS WE MADE IT!!! YAY,guys yay,2,"[guy, yay]"
126269,submission,gzl2ec,2020,"PSA: Immigration to Canada, Australia, NZ Hell...",psa immigration canada australia nz hello duba...,1342,"[psa, immigration, canada, australia, nz, hell..."
127899,submission,147gsfl,2023,How to reach people who are asking for money/g...,reach people asking money grocery experience t...,26,"[reach, people, ask, money, grocery, experienc..."
128041,submission,12pqx6m,2023,How do you plan to spend your Eid holiday? Any...,plan spend eid holiday fun activities trips li...,28,"[plan, spend, eid, holiday, fun, activity, tri..."
128388,submission,vtelex,2022,"Hi everyone, I'm currently looking for jobs te...",hi currently looking jobs temporary permanent ...,12,"[hi, currently, look, job, temporary, permanen..."
...,...,...,...,...,...,...,...
129949,submission,14djkvs,2023,Help‼️Uni qualification equivalency letter for...,help uni qualification equivalency letter mast...,37,"[help, uni, qualification, equivalency, letter..."
130007,submission,14d2uk9,2023,PSA: Villas are at high risk of fire during th...,psa villas high risk fire summer months friend...,48,"[psa, villas, high, risk, fire, summer, month,..."
130009,submission,14d0c29,2023,"Daily Random Discussion Thread Hello,\n\nThis ...",daily random discussion thread hello daily ran...,45,"[daily, random, discussion, thread, hello, dai..."
130040,submission,14cieby,2023,Need help with applying for residency - confus...,need help applying residency confused order st...,68,"[need, help, apply, residency, confused, order..."


## **Remove Short Text Entries**

In [23]:
#redo word_count
data['word_count'] = data['clean_text'].apply (lambda text: len(text.split()))

data.loc[sample_rows].sort_values(by='word_count', ascending = False)

#select only rows with more than 1 word
corpus = data[data['word_count'] > 2]

corpus.shape

(100372, 7)

In [26]:
filename = 'Data/corpus_v5.csv'

def export_csv():
    '''
    export pre-processed data to CSV
    '''
    corpus.to_csv(filename, index_label = 'index', quoting = csv.QUOTE_ALL, header = True)

export_csv()